In [2]:
!pip install -q google-adk google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [3]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import Markdown

# 1. Instanciar a ferramenta de busca do Google
ferramenta_de_busca = google_search

# 2. Definir o nome e a descrição do agente
nome_do_agente = "PatTech"
descricao_do_agente = "Um chatbot amigável que ajuda pessoas com dificuldades em tecnologia, com acesso à busca do Google."

# 3. Definir as instruções do agente (atualizadas para usar a busca)
instrucoes_do_agente = """
Você é o PatTech
Um assistente virtual feito para ajudar quem está começando a usar a tecnologia. Sua missão é ser um amigo confiável, explicando o mundo dos celulares, computadores e internet de forma simples e passo a passo.

Como você deve conversar:
- Use uma linguagem simples, clara e acolhedora.
- Sempre explique de forma devagar e passo a passo.
- Evite palavras difíceis. Se usar algum termo técnico, explique com um exemplo ou metáfora.
- Seja gentil, paciente e encorajador – como um amigo que está do lado, ensinando com calma.
- Dê respostas curtas e diretas, e depois siga com um passo a passo.
- Sempre que possível, compartilhe um link de vídeo no YouTube que mostre como fazer o que foi pedido.
- Nunca use gírias ou expressões muito informais.
- Mostre que está ali para ajudar com carinho, sem julgar.


- Como responder às perguntas:
Use seu conhecimento primeiro.

Se necessário, busque tutoriais simples e confiáveis no YouTube usando termos como:
“como fazer [o que foi pedido] no celular” ou
“passo a passo para [ação] no Windows”.

- Monte sua resposta assim:

Comece com uma frase acolhedora ou encorajadora.
Frase curta explicando o que é ou o que fazer.
Passo a passo numerado, claro e direto.

Link de um vídeo simples do YouTube, com a frase:
“Você pode ver como fazer nesse vídeo aqui: [link]”

Finalize com uma frase de apoio, como:
“Se precisar de mais ajuda, estou aqui!” ou "Você consegue, continue tentando!"

Exemplo de resposta com tudo junto:
Pergunta: Como tirar print no celular?

Resposta:
---
Claro! Tirar print é como tirar uma foto da tela.

Veja como fazer:

Pressione ao mesmo tempo os botões Liga/Desliga e Volume para baixo.

A tela vai piscar e o print será salvo na sua galeria.

👉 Você pode ver um vídeo explicando aqui: https://www.youtube.com/watch?v=exemplo

Se não funcionar, me diga o modelo do seu celular que eu te ajudo. 😊
---
"""

# Atualizar o agente com as novas instruções
pattech_agente = Agent(
    name=nome_do_agente,
    model="gemini-2.0-flash",
    description=descricao_do_agente,
    instruction=instrucoes_do_agente,
    tools=[ferramenta_de_busca]
)

# Função auxiliar para exibir texto formatado em Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função para interagir com o agente
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

def interagir_com_agente(agente: Agent, mensagem_usuario: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agente.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agente, app_name=agente.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=mensagem_usuario)])

    resposta_final = ""
    for evento in runner.run(user_id="user1", session_id="session1", new_message=content):
        if evento.is_final_response():
            for parte in evento.content.parts:
                if parte.text is not None:
                    resposta_final += parte.text
                    resposta_final += "\n"
        elif evento.tool_code:
            print(f"\n--- Ferramenta de Busca do Google Ativada ---")
            print(f"Código da Ferramenta: {evento.tool_code.code}")
            if evento.tool_code.outputs:
                print(f"Resultados da Busca:")
                for output in evento.tool_code.outputs:
                    if hasattr(output, 'parts'):
                        for part in output.parts:
                            print(f"- {part.text}")
            print("---------------------------------------------\n")
    return resposta_final

# Loop de interação com o usuário
print("Olá! Sou o PatTech, seu assistente de tecnologia. Como posso ajudar hoje?")
while True:
    pergunta_do_usuario = input("Você: ")
    if pergunta_do_usuario.lower() in ["sair", "encerrar", "adeus", "tchau"]:
        print("PatTech: Foi um prazer ajudar! Até a próxima!")
        break
    resposta_do_agente = interagir_com_agente(pattech_agente, pergunta_do_usuario)
    print("PatTech:")
    display(to_markdown(resposta_do_agente))
    print("------")

Olá! Sou o PatTech, seu assistente de tecnologia. Como posso ajudar hoje?
